In [1]:
# Create monthly report url

# To run this directly from the command line, 
# jupyter nbconvert --execute "USDA Monthly Report.ipynb" --to asciidoc

from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import urllib.parse

report_date = datetime.now().date()

# end = datetime.strptime(report_date, '%m/%d/%Y')
# start = date(report_date.year - 1, 1, 1)
start = date(report_date.year, 1, 1)

last_attended_cutoff = report_date - relativedelta(months=3)

donatedFoodPounds  = 147642    # From https://docs.google.com/spreadsheets/d/1lMoOrWVFC-Z0FnySEQQiuVBi2IzmAyDnhZhCCsuiYnU/edit?usp=sharing
donatedFoodValue = 1.93 * donatedFoodPounds

import config
base_url = config.church_domain_url + r"/events/reports/243292484#/"
# 361084354  Community Meal
# base_url = config.church_domain_url + r"/events/reports/361084354#/"


import urllib.parse
params = {
    'start_date': urllib.parse.quote(start.strftime('%m/%d/%Y'), safe='/'),
    'end_date': urllib.parse.quote(report_date.strftime('%m/%d/%Y'), safe='/'),
    'event_ids': '2949650',
    'attendance_status': 'attended',
    'attendance_operator': 'any',
    'page': 'spreadsheet',
}

print("Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.")
print(base_url + urllib.parse.urlencode(params))

Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.
https://newmarketchurch.breezechms.com/events/reports/243292484#/start_date=01%2F01%2F2024&end_date=10%2F25%2F2024&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet


In [2]:
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/events/reports/243292472#/&start_date=10%2F01%2F2023&end_date=10%2F31%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet

attendancereport = r"C:\Users\ralph\Downloads\FoodPantry-attendance-10-25-2024.xlsx"
peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-10-25-2024.xlsx"

import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
attendance = pd.read_excel(attendancereport)

# drop the Total row
attendance = attendance[attendance['Breeze ID'].notna()]

attendance['Breeze ID'] = attendance['Breeze ID'].astype(int)
attendance.fillna(0, inplace=True)

people = pd.read_excel(peopleexport, dtype={'Breeze ID': 'int'})
print('{count} people in input.'.format(count = len(attendance.index)))
# print(attendance.columns)

307 people in input.


In [3]:
# Iterate through each row, looking for the last date attended.
results = []
for index, row in attendance.iterrows():
    for col in attendance.columns:
        if row[col] == 'X' and datetime.strptime(col, '%b %d, %Y').date() < last_attended_cutoff:
            # Check if there are no 'X' in columns to the right
            
            if 'X' not in row[attendance.columns.get_loc(col) + 1:].values:
                results.append((row['First Name'], row['Last Name'], col))
                break

# Display the results
print("No longer attending...")
for result in results:
    print(f"{result[0]} {result[1]} last attended {result[2]}")
    # print(result)


No longer attending...
Jess Abbott last attended Feb 8, 2024
Ashley Aloisio last attended Feb 8, 2024
Michele Ambrose last attended May 30, 2024
Karen Anderson last attended Mar 21, 2024
Samantha Archambault last attended Jul 11, 2024
Carol Bean last attended Jun 27, 2024
Meghan Behen last attended Jun 27, 2024
Christine Bergeron last attended Mar 21, 2024
Heather Berube last attended Jan 11, 2024
Mark Bouchard last attended Jun 27, 2024
Telesha Brigham last attended May 16, 2024
Sarah L. Brown last attended Feb 15, 2024
Patrick Butler last attended Apr 25, 2024
Teri S. Butler last attended Apr 25, 2024
Dana Carey last attended Mar 7, 2024
Vincent Carnevale last attended Jan 4, 2024
Gail Champagne last attended Apr 18, 2024
Janine Chiampa last attended Mar 21, 2024
Deb Clough last attended Jul 18, 2024
Thomas Conway last attended Apr 18, 2024
Felicia Currie last attended May 30, 2024
Megan Delanoy last attended Jun 6, 2024
Kaitlin Dickens last attended Apr 11, 2024
Damaris Dominguez la

In [4]:
# Keep only the columns we use.

attendance = attendance.drop([
    'First Name', 
    'Last Name', 
    'Person Totals'
], axis = 1)

dates = attendance.columns[attendance.columns != 'Breeze ID']

family_size = {k: v for k, v in zip(people['Breeze ID'], people['How many people live in your household? (including yourself)'])}
seniors = {k: v for k, v in zip(people['Breeze ID'], people['Total number of people aged 60 or older in Household?'])}


In [5]:
first_dates = attendance.drop(['Breeze ID'], axis=1).ne(0).idxmax(axis=1).value_counts()

for date in dates:
    if date not in first_dates.index:
        first_dates.at[date] = 0

last_dates = attendance[attendance.drop(['Breeze ID'], axis=1).columns[::-1]].ne(0).idxmax(axis=1).value_counts()
for date in dates:
    if date not in last_dates.index:
        last_dates.at[date] = 0

new_shoppers = pd.DataFrame({'date': first_dates.index, 'count': first_dates.values})
absent_shoppers = pd.DataFrame({'date': last_dates.index, 'count': last_dates.values})

In [6]:
import math

# Estimate the number of seniors at the Sunrise Center based on input from the Pantry Volunteer there.
numberOfSeniors = 25

people_fed = {}
families_fed = {}
seniors_fed = {}
for date in dates:
    datesum = 0
    datecount = 0
    seniorcount = 0
    for _, row in attendance.iterrows():
        if row[date] == 'X':
            datecount = 1 + datecount
            datesum += family_size[row['Breeze ID']] if not math.isnan(family_size[row['Breeze ID']]) else 0
            seniorcount += seniors[row['Breeze ID']] if not math.isnan(seniors[row['Breeze ID']]) else 0
    families_fed[date] = datecount
    people_fed[date] = datesum + numberOfSeniors if datesum > 0 else 0
    seniors_fed[date] = seniorcount + numberOfSeniors if datesum > 0 else 0


In [7]:
# Get additional people fed at Community Meals
# Look for an "Attendance: xx" number in the meal event description.

import re

# Set up Breeze API wrapper
from breeze_chms_api import breeze

# Initialize API 
import config

breeze_api = breeze.breeze_api(breeze_url=config.church_domain_url, api_key=config.breeze_api_key)

MealsPerWeek = 15

# Assume the number of seniors matches the ratio we observe on shopping days.
total_people = sum(people_fed.values())
total_seniors = sum(seniors_fed.values())

# Find the meal event by name and date.
events = breeze_api.list_events(start=start, end=report_date, category_id=55532, details=1)
mealevents = [e for e in events if e['name'] == 'Community Meal!']

# Look for an attendance number in the event description.
pattern = r"[A|a]ttendance\D*(?P<number>\d*)"
for m in mealevents:
    mealdate = datetime.strptime(m['start_datetime'], '%Y-%m-%d %H:%M:%S').date().strftime('%b %d, %Y')
    dates = dates.append(pd.Index([mealdate]))
    match = re.search(pattern, m['details']['event_description'])
    fed = match.groupdict()['number']
    people_fed[mealdate] = round(int(fed) / MealsPerWeek)
    families_fed[mealdate] = 0
    seniors_fed[mealdate] = round(people_fed[mealdate] * total_seniors / total_people)
    # print(m)

In [8]:

output = pd.DataFrame({'Date': dates, 
                       'People Fed': people_fed.values(), 
                       'Families Fed': families_fed.values(),
                       'Seniors Fed': seniors_fed.values(),
                       'Value': [round(p * donatedFoodValue / total_people, 2) for p in people_fed.values()] })

In [9]:
people_fed_file = r"C:\Users\ralph\Downloads\People Fed.csv"
output.to_csv(people_fed_file, index=False)

new_shoppers_file = r"C:\Users\ralph\Downloads\New Shoppers.csv"
new_shoppers.to_csv(new_shoppers_file, index=False)

absent_shoppers_file = r"C:\Users\ralph\Downloads\Absent Shoppers.csv"
absent_shoppers.to_csv(absent_shoppers_file, index=False)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ralph\\Downloads\\People Fed.csv'